In [1]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import math

In [2]:
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)
navegador.get("https://esaj.tjsp.jus.br/cjpg/")

In [3]:
# Aguarde até que o botão seja visível e clicável
wait = WebDriverWait(navegador, 10)
botao_procurar = wait.until(EC.element_to_be_clickable((By.ID, "botaoProcurar_assunto")))
botao_procurar.click()

In [4]:
# Role a página até que o elemento seja visível
elemento1 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="assunto_tree_node_10064"]')))
navegador.execute_script("arguments[0].scrollIntoView();", elemento1)

elemento1.click()

In [5]:
# Role a página até que o elemento seja visível
elemento2 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="assunto_tree_node_6684"]')))
navegador.execute_script("arguments[0].scrollIntoView();", elemento2)

elemento2.click()

In [6]:
# Reduza o zoom da página para tornar o terceiro elemento visível
navegador.execute_script("document.body.style.zoom='80%'")

# Aguarde um curto intervalo de tempo antes de clicar no elemento
time.sleep(0.5)

# Clique no terceiro elemento usando execute_script
elemento_3 = navegador.find_element(By.XPATH, '//*[@id="assunto_treeSelectContainer"]/div[3]/table/tbody/tr/td/input[1]')
navegador.execute_script("arguments[0].click();", elemento_3)

In [7]:
# Dá pra voltar até 2009
ano = '2018'

In [8]:
elemento4 = navegador.find_element(By.XPATH, '//*[@id="iddadosConsulta.dtInicio"]')
elemento4.send_keys('01/01/' + ano)
    
elemento5 = navegador.find_element(By.XPATH, '//*[@id="iddadosConsulta.dtFim"]')
elemento5.send_keys('31/03/' + ano)
# março 31
# junho 30
# setembro 30

In [9]:
# Clique no botão submit usando execute_script
botao_submit = navegador.find_element(By.XPATH, '//*[@id="pbSubmit"]')
navegador.execute_script("arguments[0].click();", botao_submit)

In [10]:
html = navegador.page_source
soup = BeautifulSoup(html, 'html.parser')

texto_geral = soup.find('div', attrs={'id': 'resultados'}).find('td')

strong_text = texto_geral.find('strong').get_text().strip()

# Achar o texto total dentro do <td> tag
td_text = texto_geral.get_text().strip()

# Substituir o texto do <strong> tag dentro do <td> text
final_text = td_text.replace(strong_text, '').strip()

# Remove "de " do texto final
#final_text = final_text.replace("de ", "").strip()

# Usa expressão regular para limpar o texto e deixar apenas os números
final_text = re.sub(r'\D', '', final_text)

numero_casos = int(final_text)
numero_maximo_paginas = numero_casos/10
numero_maximo_paginas = math.ceil(numero_maximo_paginas)

In [ ]:
info_decisoes = []

for pagina_numero in range(1, numero_maximo_paginas + 1):
    print(str(pagina_numero)+"/"+str(numero_maximo_paginas))
    
    html = navegador.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    element = WebDriverWait(navegador, 120).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="divDadosResultado"]/table/tbody/tr[1]/td[2]/table/tbody/tr[1]/td'))  # Insira o XPath do elemento desejado aqui
    )
    
    lista_de_tr = soup.find('div', attrs={'id': 'divDadosResultado'}).find('table').find('tbody').find_all('tr', class_='fundocinza1')
    
    for tr in lista_de_tr:
        infos = tr.find("tbody").find_all("tr")

        num_processo = infos[0].find("a").text.strip() if infos[0].select_one('a') else ''
        classe = infos[1].select_one('td > strong').next_sibling.strip() if infos[1].select_one('td > strong') else ''
        assunto = infos[2].select_one('td > strong').next_sibling.strip() if infos[2].select_one('td > strong') else ''
        magistrado = infos[3].select_one('td > strong').next_sibling.strip() if infos[3].select_one('td > strong') else ''
        comarca = infos[4].select_one('td > strong').next_sibling.strip() if infos[4].select_one('td > strong') else ''
        foro = infos[5].select_one('td > strong').next_sibling.strip() if infos[5].select_one('td > strong') else ''
        vara = infos[6].select_one('td > strong').next_sibling.strip() if infos[6].select_one('td > strong') else ''
        data = infos[7].select_one('td > strong').next_sibling.strip() if infos[7].select_one('td > strong') else ''
        if len(infos) > 8:
            ementa_maior = infos[8].find("div", style='display: none;').text.strip() if infos[8].find("div", style='display: none;') else ''
        else:
            ementa_maior = ''
        
        info_decisoes.append([num_processo, classe, assunto, magistrado, comarca, foro, vara, data, ementa_maior])#, new_window_url])
    
    # Verificar se a página atual é menor ou igual ao número máximo de páginas definido
    if pagina_numero == 1:
        numero_pagina = WebDriverWait(navegador, 120).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="resultados"]/table[1]/tbody/tr[1]/td[2]/div/a[1]'))  # Insira o XPath do elemento desejado aqui
        )
        numero_pagina.click()
    else:
        if pagina_numero == 2:
            numero_pagina = WebDriverWait(navegador, 120).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="resultados"]/table[1]/tbody/tr[1]/td[2]/div/a[3]'))  # Insira o XPath do elemento desejado aqui
            )
            numero_pagina.click()
        else:
            if pagina_numero == numero_maximo_paginas - 1:
                numero_pagina = WebDriverWait(navegador, 120).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="resultados"]/table[1]/tbody/tr[1]/td[2]/div/a[5]'))  # Insira o XPath do elemento desejado aqui
                )
                numero_pagina.click()
            elif pagina_numero < numero_maximo_paginas:
                numero_pagina = WebDriverWait(navegador, 180).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="resultados"]/table[1]/tbody/tr[1]/td[2]/div/a[4]'))  # Insira o XPath do elemento desejado aqui
                )
                numero_pagina.click()        
    time.sleep(5)

1/346


In [ ]:
decisoes_df = pd.DataFrame(info_decisoes)
decisoes_df.columns = ["Nº do Processo", "Classe", "Assunto", "Magistrado", "Comarca", "Foro", "Vara", "Data", "Ementa"]#, "Link"]

agora = datetime.now().strftime("%Y%m%d%H%M%S")
nome_arquivo = "esaj1-" + ano + "-1-" + agora + ".xlsx"
decisoes_df.to_excel(nome_arquivo, index=False)